# K-Nearest Neighbors Classifier - the Random Dataset

### Introducing the database

The database for this example is randomly generated. Generating datasets is an important skill to have. This is the best way to explore the algorithms themelves without having to worry about encoding, standardizing, dealing with outliers and all sorts of preprocessing that comes with a real-world dataset. Another advantage of randomly generated datasets is that you have full control over the data. You are free to choose the number of features, the number of classes, the way the points are distributed, everything really! 

I wish you to have lots of fun with the exercise and keep experimenting!

### Importing the necessary libraries

In [ ]:
from sklearn.datasets import make_blobs

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score

from sklearn.model_selection import GridSearchCV

from mlxtend.plotting import plot_decision_regions

import time

import numpy as np

### Generating the database

In [ ]:
inputs, target = make_blobs(n_samples = 1000, 
                            centers = [(-3, 3), (0, 0), (2, 2)], 
                            random_state = 365)

In [ ]:
inputs.shape, target.shape

In [ ]:
data = pd.DataFrame(data = inputs, columns = ['Feature 1', 'Feature 2'])
data['Target'] = target
data

### Plotting the database

In [ ]:
knn_palette = sns.color_palette(['#000C1F', '#29757A', '#FF5050'])
knn_palette

In [ ]:
sns.set()

plt.figure(figsize = (16, 9))

sns.scatterplot(x = 'Feature 1', y = 'Feature 2', 
                data = data, 
                hue = 'Target', 
                palette = knn_palette,
                markers = [',', '^', 'P'],
                style = 'Target', 
                s = 100);

### Visualizing the distribution of the points

In [ ]:
sns.set()

sns.jointplot(x = 'Feature 1', y = 'Feature 2', 
              data = data, 
              hue = 'Target', 
              palette = knn_palette,
              height = 10,
              s = 100, 
              legend = True);

### Creating a train-test-split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs, target, 
                                                    test_size=0.2, 
                                                    random_state=365, 
                                                    stratify = target)

### Creating the model

In [ ]:
clf = KNeighborsClassifier(n_neighbors = 30, weights = 'uniform')

clf.fit(x_train, y_train)

### Predicting a sample

In [ ]:
feature_1 = -0.18
feature_2 = 3.2

In [ ]:
clf.predict([[feature_1, feature_2]])

In [ ]:
neighbors = clf.kneighbors([[feature_1, feature_2]])
neighbors

### Visualizing the neighbors

In [ ]:
sns.set()

plt.figure(figsize = (16, 9))

sns.scatterplot(x = x_train[:, 0], y = x_train[:, 1],
                hue = y_train, 
                palette = knn_palette,
                markers = [',', '^', 'P'],
                style = y_train, 
                s = 100, 
                legend = True);

sns.scatterplot(x = [feature_1], y = [feature_2], 
                style = [feature_2],
                markers = ['o'],
                s = 100,
                legend = False);

plot_x_train = []
plot_y_train = []

for i in neighbors[1]:
    plot_x_train.append(x_train[i, 0])
    plot_y_train.append(x_train[i, 1])

plt.scatter(plot_x_train, 
            plot_y_train, 
            s=200, facecolors='none', edgecolors='r');

### Getting the parameters of the model

In [ ]:
clf.get_params()

### Drawing the decision regions

In [ ]:
x_train.shape

In [ ]:
start = time.time()

plt.figure(figsize = (16, 9))

plot_decision_regions(X = x_train, y = y_train,
                      X_highlight = np.array([[feature_1, feature_2]]),
                      clf = clf,
                      markers = [',', '^', 'P'],
                      colors = '#000c1f,#29757a,#ff5050',
                      scatter_kwargs = {'s':100, 'edgecolor':'white', 'alpha':1},
                      legend = 0)

plt.xlabel('Feature 1')
plt.ylabel('Feature 2');

end = time.time()

print(f'Time elapsed: {round(end - start, 1)} seconds')

In [ ]:
np.array([[feature_1, feature_2]]).shape

### Getting the error rates of a set of models

In [ ]:
error_uniform = []
error_distance = []

k_range = range(1, 51)

for k in k_range:
    
    clf = KNeighborsClassifier(n_neighbors = k, weights = 'uniform')
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)
    error_uniform.append(1 - accuracy_score(y_test, predictions))
    
    clf = KNeighborsClassifier(n_neighbors = k, weights = 'distance')
    clf.fit(x_train, y_train)
    predictions = clf.predict(x_test)
    error_distance.append(1 - accuracy_score(y_test, predictions))

### Plotting the error rates as a function of the number of neighbors

In [ ]:
plt.figure(figsize = (16, 9))

plt.plot(k_range, error_uniform, c = 'blue', linestyle = 'solid', 
         marker = 'o', markerfacecolor = 'red',  label = 'Error uniform');
plt.plot(k_range, error_distance, c = 'green', linestyle = 'dashed', 
         marker = 'o', markerfacecolor = 'red', label = 'Error distance');

plt.legend()

plt.xlabel('K-value')
plt.ylabel('Error rate');

### Choosing a set of parameters to test

In [ ]:
parameters = {'n_neighbors':range(1, 51), 
              'weights':['uniform', 'distance']}

### Create an instance of the GridSearchCV class

In [ ]:
grid_search = GridSearchCV(estimator = KNeighborsClassifier(), 
                           param_grid = parameters, 
                           scoring = 'accuracy')

In [ ]:
grid_search.fit(x_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
clf = grid_search.best_estimator_

In [ ]:
clf

In [ ]:
grid_search.best_score_

### Make predictions on the test dataset

In [ ]:
y_test_pred = clf.predict(x_test)
y_test_pred.shape

### Construct the confusion matrix

In [ ]:
sns.reset_orig()

ConfusionMatrixDisplay.from_predictions(
    y_test, y_test_pred,
    labels = clf.classes_,
    cmap = 'magma'
);

### Print out the classification report

In [ ]:
print(classification_report(y_test, y_test_pred, target_names = ['0', '1', '2']))